# Import everything

In [1]:
import os
import skimage.io as io
import skimage.transform as trans
import random
import h5py
import re
import matplotlib
import rasterio
import numpy as np
from rasterio.plot import show, show_hist
from rasterio.merge import merge as rastermerge
from matplotlib import pyplot
%matplotlib inline
from keras import metrics
from keras import backend as K
from keras.initializers import *
from keras.layers import *
from keras.models import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from importlib import reload
from keras.callbacks import EarlyStopping, TensorBoard
from keras.models import load_model
from keras.models import save_model

Using TensorFlow backend.


# Set locations

In [2]:
PREDICTIONS_DIR = "/Users/ciaran/data/imagery/tifs/blue_anchor/predictions"
IMAGES_TO_PREDICT_DIR = "/Users/ciaran/data/imagery/tifs/blue_anchor/train/normalised_images"

# Load model

In [3]:
model = load_model('/Users/ciaran/dev/dsa_project/models/1e-5_subset_2/model')

# Figure out all cells that have been predicted already

In [4]:
predictedImages = [file for file in os.listdir(PREDICTIONS_DIR)]

# Predict remaining cells

In [ ]:
for file in os.listdir(IMAGES_TO_PREDICT_DIR):
            if file.endswith(".tif") and file not in predictedImages:
                imageToPredict = rasterio.open(os.path.join(IMAGES_TO_PREDICT_DIR, file)).read()
                imageToPredict = np.moveaxis(imageToPredict, 0, -1)
                imageToPredict = np.array([imageToPredict])
                prediction = model.predict(imageToPredict)[0]
                metadata = rasterio.open(os.path.join(IMAGES_TO_PREDICT_DIR, file)).meta
                metadata['count'] = 1
                metadata['dtype'] = 'float32'
                prediction = np.moveaxis(prediction, -1, 0)
                with rasterio.open(os.path.join(PREDICTIONS_DIR, file), "w", **metadata) as dest:
                    dest.write(prediction)